## This pipeline processes VCF files with VEP primarily to identify splice-altering variants using multiple tools, including MES (MaxEntScan), SpliceAI, dbscSNV, SQUIRLS, and CADD. The workflow involves preprocessing and splitting VCFs, running SpliceAI predictions, and recombining filtered results into a final dataset. The output provides a ranked list of potential splice-altering variants, facilitating downstream analysis and variant interpretation.

In [1]:
import csv
import pandas as pd
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
import argparse

In [ ]:
# Variant Effect Predictor (VEP) Splice Detection Pipeline
# This script runs VEP with splice detection plugins to analyze VCF files.

# Define paths and variables (modify as needed)
VEP_DIR = "/path/to/ensembl-vep"
GENOME_DIR = "/path/to/genome_data"
SAMPLE = "sample_id"
SAMPLE_DIR = f"/path/to/sample_data/genotype_{SAMPLE}"
OUTPUT_FILE = f"{SAMPLE_DIR}/{SAMPLE}.VEP.vcf.gz"

# Bash script for VEP annotation with splice detection plugins
vep_script = f"""
#!/bin/bash
#SBATCH --job-name=VEP-splice
#SBATCH --mem=400G
#SBATCH --output=%x-%A_%a.out

module load tabix
source /path/to/miniconda3/bin/activate /path/to/envs/my_env

# Define input VCF file
INPUT={SAMPLE_DIR}/{SAMPLE}.nodups.vcf.gz

# Run VEP with various plugins
{VEP_DIR}/vep --input_file $INPUT \\
       --output_file {OUTPUT_FILE} \\
       --species homo_sapiens \\
       --vcf \\
       --fork 4 \\
       --cache \\
       --minimal \\
       --pick \\
       --force_overwrite \\
       --compress_output bgzip \\
       --plugin MaxEntScan,{VEP_DIR}/Plugins/MaxEntScan/fordownload \\
       --plugin SpliceAI,snv={GENOME_DIR}/spliceai_scores.masked.snv.hg38.vcf.gz,indel={GENOME_DIR}/spliceai_scores.masked.indel.hg38.vcf.gz \\
       --plugin SpliceRegion \\
       --plugin GeneSplicer,{VEP_DIR}/Plugins/GeneSplicer/genesplicer,{VEP_DIR}/Plugins/GeneSplicer/human \\
       --plugin SpliceVault,file={VEP_DIR}/Plugins/SpliceVault/SpliceVault_data_GRCh38.tsv.gz \\
       --plugin dbscSNV,{VEP_DIR}/Plugins/dbscSNV/dbscSNV1.1_GRCh38.txt.gz \\
       --plugin CADD,snv={VEP_DIR}/Plugins/CADD/whole_genome_SNVs.tsv.gz,indels={VEP_DIR}/Plugins/CADD/gnomad.genomes.r4.0.indel.tsv.gz \\
       --dir_cache {VEP_DIR}/CACHEDIR \\
       --dir_plugins {VEP_DIR}/Plugins \\
       --fasta {GENOME_DIR}/hg38.fa

echo "Processing sample: {SAMPLE}"

# Process output VCF file
cd {SAMPLE_DIR}
bgzip -d {SAMPLE}.VEP.vcf.gz
grep -v '^#' {SAMPLE}.VEP.vcf > {SAMPLE}.noHeader.txt
tail -n +2 {SAMPLE}.noHeader.txt > {SAMPLE}.finalFilter.txt
bgzip {SAMPLE}.VEP.vcf

# Extract relevant fields for splice annotation
cut -f8 {SAMPLE}.finalFilter.txt > {SAMPLE}.onlyINFO.txt
cut -f1 {SAMPLE}.finalFilter.txt > {SAMPLE}.onlyHeaders.txt
cut -f2 {SAMPLE}.finalFilter.txt > {SAMPLE}.onlyLoc.txt

# Extract last set of fields related to SpliceAI annotation
awk -F '|' '{{ for (i = NF - 24; i <= NF; i++) {{ printf("%s", $i); if (i < NF) printf("|"); }} printf("\\n"); }}' {SAMPLE}.onlyINFO.txt > {SAMPLE}.spliceAI_only.txt

# Merge extracted fields into a final annotated file
paste {SAMPLE}.onlyHeaders.txt {SAMPLE}.onlyLoc.txt {SAMPLE}.spliceAI_only.txt > {SAMPLE}.merged_VEP.txt

# Clean up intermediate files
rm {SAMPLE}.onlyHeaders.txt {SAMPLE}.finalFilter.txt {SAMPLE}.onlyLoc.txt {SAMPLE}.spliceAI_only.txt {SAMPLE}.onlyINFO.txt {SAMPLE}.noHeader.txt

# Remove empty annotations
awk -F'\\t' '$3 !~ /\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|\\|/' {SAMPLE}.merged_VEP.txt > {SAMPLE}.filtered_VEP.txt
"""

# Save the script to a file (optional)
script_path = f"{SAMPLE_DIR}/run_vep_splice.sh"
with open(script_path, "w") as f:
    f.write(vep_script)

# Print the script path for reference
print(f"Saved VEP script to: {script_path}")

In [ ]:
# SQUIRLS Annotation Pipeline
# This script runs SQUIRLS to annotate VCF files for splicing predictions.

# Define paths and variables (modify as needed)
SQUIRLS_DIR = "/path/to/squirls"
SAMPLE = "sample_id"
SAMPLE_DIR = f"/path/to/sample_data/genotype_{SAMPLE}"
OUTPUT_FILE = f"{SAMPLE_DIR}/{SAMPLE}-squirls.csv"

# Define all chromosomes for parallel processing (if applicable)
CHROMOSOMES = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10",
               "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19",
               "chr20", "chr21", "chr22", "chrX", "chrY", "chrM"]

# Bash script for SQUIRLS annotation
squirls_script = f"""
#!/bin/bash
#SBATCH --job-name=SQUIRLS-annotate
#SBATCH --mem=300G
#SBATCH --output=%x-%A_%a.out

# Select chromosome based on task array ID
CHROMOSOMES=({" ".join(CHROMOSOMES)})
chrom="{{CHROMOSOMES[$SLURM_ARRAY_TASK_ID]}}"

# Run SQUIRLS annotation
java -jar -Xms2g -Xmx300g {SQUIRLS_DIR}/squirls-cli-2.0.0.jar annotate-vcf \\
    -d {SQUIRLS_DIR} -f csv -n 10000000 \\
    {SAMPLE_DIR}/{SAMPLE}.nodups.vcf.gz \\
    {SAMPLE_DIR}/{SAMPLE}-squirls

# Filter results: remove NaN values and keep scores >= 0.2
awk -F',' '$9 != "NaN" && $9 >= 0.2' {SAMPLE_DIR}/{SAMPLE}-squirls.csv > {SAMPLE_DIR}/{SAMPLE}-squirls-filtered.csv

# Sort results by score (column 9) in descending order
sort -t',' -k9,9nr {SAMPLE_DIR}/{SAMPLE}-squirls-filtered.csv > {SAMPLE_DIR}/{SAMPLE}-squirls-sorted.csv

echo "Completed SQUIRLS annotation for sample: {SAMPLE}"

# Remove intermediate filtered file
rm {SAMPLE_DIR}/{SAMPLE}-squirls-filtered.csv

# Remove chromosome-specific VCFs after annotation
rm {SAMPLE_DIR}/{SAMPLE}-split/{SAMPLE}.$chrom.vcf.gz
"""

# Save the script to a file (optional)
script_path = f"{SAMPLE_DIR}/run_squirls.sh"
with open(script_path, "w") as f:
    f.write(squirls_script)

# Print the script path for reference
print(f"Saved SQUIRLS script to: {script_path}")


In [ ]:
%%bash
#SBATCH --job-name=spliceAI_preprocess
#SBATCH --mem=400G
#SBATCH --output=spliceAI_preprocess_%A.out

# Load environment
source /path/to/miniconda3/bin/activate /path/to/envs/my_env

# Define sample and paths
SAMPLE="sample_id"
SAMPLE_DIR="/path/to/genotyped_VCFs/genotype_$SAMPLE"
SPLIT_DIR="$SAMPLE_DIR/$SAMPLE-500"

# Ensure necessary directories exist
mkdir -p "$SPLIT_DIR"

# Start time tracking
start_time=$(date +%s)

echo "Processing SpliceAI for sample: $SAMPLE"

# Step 1: Remove duplicate variants from the VCF file
bcftools norm -d none -o "$SAMPLE_DIR/$SAMPLE.nodups.vcf.gz" "$SAMPLE_DIR/$SAMPLE.exons5000.vcf.gz"
tabix "$SAMPLE_DIR/$SAMPLE.nodups.vcf.gz"

# Step 2: Calculate total variants and determine split size
TOTAL_VARIANTS=$(bcftools view -H "$SAMPLE_DIR/$SAMPLE.nodups.vcf.gz" | wc -l)
VARIANTS_PER_FILE=$((TOTAL_VARIANTS / 500 + 1))

echo "Total Variants: $TOTAL_VARIANTS, Variants Per File: $VARIANTS_PER_FILE"

# Step 3: Split the VCF into smaller files
temp_dir=$(mktemp -d)
bcftools view -h "$SAMPLE_DIR/$SAMPLE.nodups.vcf.gz" > "$temp_dir/header.vcf"
bcftools view -H "$SAMPLE_DIR/$SAMPLE.nodups.vcf.gz" | split -l "$VARIANTS_PER_FILE" - "$temp_dir/part_"

for file in "$temp_dir"/part_*; do
    filename=$(basename "$file")
    cat "$temp_dir/header.vcf" "$file" > "$SPLIT_DIR/${filename}.vcf"
    bgzip "$SPLIT_DIR/${filename}.vcf"
    tabix -p vcf "$SPLIT_DIR/${filename}.vcf.gz"
done

rm -r "$temp_dir"

# End time tracking
end_time=$(date +%s)
runtime=$((end_time - start_time))

echo "SpliceAI Preprocessing completed in $runtime seconds."


In [ ]:
%%bash
#SBATCH --job-name=spliceAI_recombine
#SBATCH --mem=400G
#SBATCH --output=spliceAI_recombine_%A.out

# Define sample and paths
SAMPLE="sample_id"
SAMPLE_DIR="/path/to/genotyped_VCFs/genotype_$SAMPLE"
SPLIT_DIR="$SAMPLE_DIR/$SAMPLE-500"
RECOMBINED_VCF="$SAMPLE_DIR/$SAMPLE.recombined.vcf.gz"

echo "Recombining SpliceAI processed VCFs for sample: $SAMPLE"

# Step 1: Index and collect VCF file paths
vcf_files=()
for file in "$SPLIT_DIR"/part_*-OUT.vcf.gz; do
    bcftools index -f "$file"
    vcf_files+=("$file")
done

# Step 2: Concatenate all SpliceAI-processed VCFs into one
bcftools concat -a "${vcf_files[@]}" -o "$RECOMBINED_VCF"

# Step 3: Index the concatenated VCF
bcftools index -f "$RECOMBINED_VCF"

echo "VCF recombination completed."

# Step 4: Extract SpliceAI annotations
bgzip -d "$RECOMBINED_VCF"
grep -v '^#' "${RECOMBINED_VCF%.gz}" > "$SAMPLE_DIR/$SAMPLE.noHeader.txt"
tail -n +2 "$SAMPLE_DIR/$SAMPLE.noHeader.txt" > "$SAMPLE_DIR/$SAMPLE.finalFilter.txt"
bgzip "$RECOMBINED_VCF"

# Step 5: Extract relevant SpliceAI information
cut -f8 "$SAMPLE_DIR/$SAMPLE.finalFilter.txt" > "$SAMPLE_DIR/$SAMPLE.onlyINFO.txt"
cut -f1 "$SAMPLE_DIR/$SAMPLE.finalFilter.txt" > "$SAMPLE_DIR/$SAMPLE.onlyHeaders.txt"
cut -f2 "$SAMPLE_DIR/$SAMPLE.finalFilter.txt" > "$SAMPLE_DIR/$SAMPLE.onlyLoc.txt"

rm "$SAMPLE_DIR/$SAMPLE.noHeader.txt" "$SAMPLE_DIR/$SAMPLE.finalFilter.txt"

# Step 6: Extract SpliceAI values
awk -F ';' '{ for (i=1; i<=NF; i++) { if ($i ~ /^SpliceAI=/) { print substr($i, index($i, "=") + 1); next } } print "" }' "$SAMPLE_DIR/$SAMPLE.onlyINFO.txt" > "$SAMPLE_DIR/$SAMPLE.split.txt"

paste "$SAMPLE_DIR/$SAMPLE.onlyHeaders.txt" "$SAMPLE_DIR/$SAMPLE.onlyLoc.txt" "$SAMPLE_DIR/$SAMPLE.split.txt" > "$SAMPLE_DIR/$SAMPLE.merged_spliceAI.txt"

# Move results to main directory
cp "$SPLIT_DIR/$SAMPLE.merged_spliceAI.txt" "$SAMPLE_DIR/$SAMPLE.merged_spliceAI.txt"

# Step 7: Filter out SpliceAI scores below threshold (>= 0.10)
awk -F '\t' '$2 != "" && ($3 >= 0.10 || $4 >= 0.10 || $5 >= 0.10 || $6 >= 0.10)' "$SAMPLE_DIR/$SAMPLE.merged_spliceAI.txt" > "$SAMPLE_DIR/$SAMPLE.filtered_spliceAI.txt"

# Step 8: Remove duplicate lines
sort "$SAMPLE_DIR/$SAMPLE.filtered_spliceAI.txt" | uniq > "$SAMPLE_DIR/$SAMPLE.uniq.txt"

# Step 9: Convert column 3 into separate plugin fields
awk -F'\t' 'BEGIN {OFS="\t"} {gsub(/\|/, "\t", $0); print}' "$SAMPLE_DIR/$SAMPLE.uniq.txt" > "$SAMPLE_DIR/$SAMPLE.sep.txt"
awk -F'\t' '($5 >= 0.10 || $6 >= 0.10 || $7 >= 0.10 || $8 >= 0.10)' "$SAMPLE_DIR/$SAMPLE.sep.txt" > "$SAMPLE_DIR/$SAMPLE.splitF.10.txt"

# Step 10: Cleanup intermediate files
rm "$SAMPLE_DIR/$SAMPLE.onlyHeaders.txt" "$SAMPLE_DIR/$SAMPLE.onlyLoc.txt" "$SAMPLE_DIR/$SAMPLE.split.txt" \
   "$SAMPLE_DIR/$SAMPLE.merged_spliceAI.txt" "$SAMPLE_DIR/$SAMPLE.filtered_spliceAI.txt" "$SAMPLE_DIR/$SAMPLE.uniq.txt" \
   "$SAMPLE_DIR/$SAMPLE.sep.txt" "$SAMPLE_DIR/$SAMPLE.onlyINFO.txt"

echo "SpliceAI processing completed."


In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import os

# Define base directory paths (Modify as needed)
VEP_DIR = "/path/to/VEP_tables"
SQUIRLS_DIR = "/path/to/squirls"
SPLICEAI_DIR = "/path/to/splice_files"

def process_vep_output(sample_name):
    """
    Processes VEP output by extracting variant annotations, filtering results, and classifying variants.
    """

    # Define input file path
    vep_file = os.path.join(VEP_DIR, f"{sample_name}.filtered_VEP.txt")

    # Load VEP output with predefined column names
    colnames = ['chr', 'loc', 'info']
    VEP_df = pd.read_csv(vep_file, sep="\t", names=colnames)

    # Split the 'info' column into separate annotation fields
    info_columns = VEP_df['info'].str.split('|', expand=True)

    # Define expected column names
    newcols = ['chr', 'loc', 'MES_alt', 'MES_diff', 'MES_ref', 'SpliceAI_DP_AG', 'SpliceAI_DP_AL', 'SpliceAI_DP_DG',
               'SpliceAI_DP_DL', 'SpliceAI_DS_AG', 'SpliceAI_DS_AL', 'SpliceAI_DS_DG', 'SpliceAI_DS_DL',
               'SpliceAI_SYMBOL', 'SpliceRegion', 'GeneSplicer', 'SpliceVault_SpliceAI_delta', 'SpliceVault_oot_events',
               'SpliceVault_max_depth', 'SpliceVault_pos', 'SpliceVault_count', 'SpliceVault_type', 'SpliceVault_events',
               'dbscSNV_ada', 'dbscSNV_RF', 'CADD_phred', 'CADD_raw']

    # Reconstruct DataFrame with expanded annotations
    VEP_df = pd.concat([VEP_df.drop(columns=['info']), info_columns], axis=1)
    VEP_df.columns = newcols

    # Convert numeric columns
    numeric_cols = ['MES_alt', 'MES_diff', 'dbscSNV_ada', 'dbscSNV_RF', 'CADD_phred', 'CADD_raw']
    VEP_df[numeric_cols] = VEP_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # MES filtering: Identify high and low impact mutations
    conditions = [
        ((VEP_df['MES_diff'] > 1.15) & (VEP_df['MES_alt'] < 6.2)) | ((VEP_df['MES_diff'] < 0) & (VEP_df['MES_alt'] > 8.5)),
        (VEP_df['MES_alt'].isna() | VEP_df['MES_diff'].isna())
    ]
    choices = ['High', 'None']
    VEP_df['MES_outcome'] = np.select(conditions, choices, default='Low')

    # Keep only High and Low outcomes
    MES_df = VEP_df[VEP_df['MES_outcome'].isin(['High', 'Low'])][['chr', 'loc', 'MES_alt', 'MES_diff', 'MES_ref', 'MES_outcome']]

    # dbscSNV filtering
    dbsc_df = VEP_df[(VEP_df['dbscSNV_ada'].notna()) & (VEP_df['dbscSNV_RF'].notna())]
    dbsc_conditions = [(dbsc_df['dbscSNV_ada'] >= 0.6) | (dbsc_df['dbscSNV_RF'] >= 0.6)]
    dbsc_choices = ['High']
    dbsc_df['dbsc_outcome'] = np.select(dbsc_conditions, dbsc_choices, default='Low')
    dbsc_df = dbsc_df[dbsc_df['dbsc_outcome'].isin(['High', 'Low'])][['chr', 'loc', 'dbscSNV_ada', 'dbscSNV_RF', 'dbsc_outcome']]

    # CADD filtering
    CADD_df = VEP_df[VEP_df['CADD_phred'].notna()]
    CADD_conditions = [CADD_df['CADD_phred'] >= 10]
    CADD_choices = ['High']
    CADD_df['CADD_outcome'] = np.select(CADD_conditions, CADD_choices, default='Low')
    CADD_df = CADD_df[CADD_df['CADD_outcome'].isin(['High', 'Low'])][['chr', 'loc', 'CADD_phred', 'CADD_raw', 'CADD_outcome']]

    # Save intermediate results
    MES_df.to_csv(f"{VEP_DIR}/MES_df_{sample_name}.csv", index=False)
    dbsc_df.to_csv(f"{VEP_DIR}/dbsc_df_{sample_name}.csv", index=False)
    CADD_df.to_csv(f"{VEP_DIR}/CADD_df_{sample_name}.csv", index=False)

    return MES_df, dbsc_df, CADD_df


In [ ]:
def merge_annotations(sample_name, MES_df, dbsc_df, CADD_df):
    """
    Merges all filtered annotation datasets (MES, dbscSNV, CADD, SQUIRLS, SpliceAI) into a single comparison table.
    """

    # Load SQUIRLS output
    squirls_file = os.path.join(SQUIRLS_DIR, f"{sample_name}-squirls-sorted.csv")
    squirls_df = pd.read_csv(squirls_file, sep=",", header=0)
    squirls_df = squirls_df.drop_duplicates(subset=['chr', 'loc'], keep='first')
    squirls_df['chr'] = squirls_df['chr'].apply(lambda x: 'chr' + str(x))
    squirls_df['squirls_score'] = pd.to_numeric(squirls_df['squirls_score'], errors='coerce')

    squirls_conditions = [squirls_df['squirls_score'] >= 0.6]
    squirls_choices = ['High']
    squirls_df['squirls_outcome'] = np.select(squirls_conditions, squirls_choices, default='Low')
    squirls_df = squirls_df[squirls_df['squirls_outcome'].isin(['High', 'Low'])][['chr', 'loc', 'squirls_score', 'squirls_outcome']]

    # Load SpliceAI output
    spliceai_file = os.path.join(SPLICEAI_DIR, f"{sample_name}.spliceFreq.csv")
    SpliceAI_df = pd.read_csv(spliceai_file, sep="\t")
    SpliceAI_df['DS_MAX'] = SpliceAI_df[['DS_AG', 'DS_AL', 'DS_DG', 'DS_DL']].max(axis=1)

    spliceai_conditions = [SpliceAI_df['DS_MAX'] >= 0.6]
    SpliceAI_choices = ['High']
    SpliceAI_df['SpliceAI_outcome'] = np.select(spliceai_conditions, SpliceAI_choices, default='Low')
    SpliceAI_df = SpliceAI_df[SpliceAI_df['SpliceAI_outcome'].isin(['High', 'Low'])][['chr', 'loc', 'DS_MAX', 'SpliceAI_outcome']]

    # Merge all data sources
    comparison_df = reduce(lambda left, right: pd.merge(left, right, on=['chr', 'loc'], how='outer'),
                           [MES_df, dbsc_df, CADD_df, squirls_df, SpliceAI_df])

    # Count number of "High" and "Low" calls
    outcome_cols = ['MES_outcome', 'dbsc_outcome', 'CADD_outcome', 'squirls_outcome', 'SpliceAI_outcome']
    comparison_df[['count_high', 'count_low']] = comparison_df[outcome_cols].apply(lambda row: [(row == 'High').sum(), (row == 'Low').sum()], axis=1, result_type='expand')

    # Sort and save the final output
    comparison_df.sort_values(by=['count_high', 'count_low'], ascending=[False, False], inplace=True)
    comparison_df.to_csv(f"{VEP_DIR}/{sample_name}_final_output.csv", index=False)

    return comparison_df

# Execute processing
MES_df, dbsc_df, CADD_df = process_vep_output("383582")
result_df = merge_annotations("383582", MES_df, dbsc_df, CADD_df)


In [ ]:
import matplotlib.pyplot as plt

def create_bar(df, columns=['MES_outcome', 'SpliceAI_outcome', 'dbsc_outcome', 'squirls_outcome', 'CADD_outcome']):
    """
    Creates a scatter plot showing the proportion of 'High' variants detected by each splice-detection tool.
    
    Parameters:
    - df (DataFrame): The DataFrame containing variant annotation outcomes.
    - columns (list): List of column names to analyze for 'High' counts.
    """
    
    # Check if DataFrame is empty
    if df.empty:
        print("The provided DataFrame is empty.")
        return
    
    # Ensure all required columns exist in the DataFrame
    missing_cols = [col for col in columns if col not in df.columns]
    if missing_cols:
        print(f"Missing columns in DataFrame: {missing_cols}")
        return
    
    # Calculate the proportion of "High" classifications
    total_rows = len(df)
    high_proportions = [df[col].fillna("").str.count("High").sum() / total_rows for col in columns]
    
    # Define plot size
    plt.figure(figsize=(10, 6))
    
    # Scatter plot of proportions
    plt.scatter(columns, high_proportions, color='blue', s=100)
    
    # Add labels and title
    plt.xlabel('Splice Detection Software')
    plt.ylabel('Proportion of Variants Classified as Splice-Altering')
    plt.title('Comparison of Splice Detection Tools')
    
    # Adjust y-axis to show proportion values clearly
    plt.ylim(0, 1)  
    plt.xticks(rotation=45)
    
    # Custom labels for clarity (matching software names)
    custom_labels = ['MES', 'SpliceAI', 'dbscSNV', 'SQUIRLS', 'CADD']
    plt.xticks(ticks=range(len(columns)), labels=custom_labels, rotation=45)
    
    # Add grid for better readability
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Show plot
    plt.show()

# Run the function on the processed result DataFrame
create_bar(result_df)
